<a href="https://colab.research.google.com/github/Filippo-Tombari/PdeGraph/blob/main/Obstacle.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import the necessary packages

In [1]:
from google.colab import drive
drive.mount('/content/drive')
import sys
sys.path.append("/content/drive/MyDrive/tesi/PdeGraph")
import os
os.chdir("/content/drive/MyDrive/tesi/PdeGraph")

Mounted at /content/drive


In [3]:
import install
install.pytorchgeo()

Pytorch geometric installed.


In [4]:
install.fenics()

FEniCS installed.


In [4]:
import numpy as np
import pickle
import functional
from functional import asfield, plot, L2, buildconnectivity
import gnns
import dolfin
import torch
import torch.nn.functional as F
from torch_geometric.loader import NeighborSampler
import torch.optim as optim

# Loading and preparation of the data

In [5]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [6]:
mesh_train = dolfin.cpp.mesh.Mesh("files/obstacleDFG.xml")
edge_index_train = buildconnectivity(mesh_train)
edge_index_train = torch.t(torch.from_numpy(edge_index_train.astype('int32')).long()).to(device)

Training data parameter: 1.5

In [7]:
with open("/content/drive/MyDrive/tesi/obstacle/files/train_set.pkl", "rb") as fp:   #Pickling
  train_set_new = pickle.load(fp)
  train_set_new = [data.float() for data in train_set_new]
train_loader = NeighborSampler(edge_index_train, node_idx=None,
                               sizes=[7, 6, 5], batch_size=train_set_new[0].size()[0])
valid_loader = NeighborSampler(edge_index_train, node_idx=None,
                                sizes=[-1], batch_size=train_set_new[0].size()[0])
for bs, id, a in train_loader:
  batch_size_train = bs
  n_id_train = id
  adjs_train = a#`adjs` holds a list of `(edge_index, e_id, size)` tuples

In [8]:
dt = 2e-2
train_set = [torch.cat((train_set_new[i],torch.full((train_set_new[0].shape[0],1), dt*i)), dim = 1) for i in range(len(train_set_new))]
#choose a specific time window for the training set in order to capture the phenomenon that we want to predict
train_set = train_set[200:300] # we choose the window [4s,6s]
train_set = torch.stack(train_set).to(device) #now the training set has size sample size x nodes x features

# Training

In [9]:
l2   = L2(mesh_train).float() # L2 norm for scalar functions
lv22 = lambda v: l2(v[:,:,0].to(device)).pow(2).float() + l2(v[:,:,1].to(device)).pow(2).float()
lv2  = lambda v: lv22(v).sqrt().float() # L2 norm for vectorial functions
def loss(output, target):
  return (lv2(target - output) / lv2(target)).mean().float()

Calling FFC just-in-time (JIT) compiler, this may take some time.
Calling FFC just-in-time (JIT) compiler, this may take some time.
Calling FFC just-in-time (JIT) compiler, this may take some time.
Calling FFC just-in-time (JIT) compiler, this may take some time.
Calling FFC just-in-time (JIT) compiler, this may take some time.


In [21]:
#def ncumsum(u,n):
#  res = u
#  for i in range(u.shape[0] - n):
#    res[i+n] = 
train_set[:,:,2].shape

torch.Size([100, 8239])

In [24]:
gnn = gnns.SAGE(3,256,3).to(device)
valid_size = 25
train_size = train_set.shape[0] - valid_size
train = train_set[:train_size,:,:].to(device)
valid = train_set[train_size:].to(device)
batch_size = 75
#learningrate = 1e-1
#optimizer = optim.Adam(gnn.parameters(), lr=learningrate)
optimizer = optim.LBFGS(gnn.parameters())
model_chk_path = 'checkpoints/obstacle_chk.pt'
rollout_train_loss = []
mse_min = 10000
early_stopping = 0
epochs = 100 
t = 1 # current epoch
done = False
while not done:
      rollout_train_loss.clear()
      for i in range(0,train_size,batch_size):
        train_loss = 0
        # training
        def closure():
          optimizer.zero_grad()
          # forward pass
          integrating = torch.stack([gnn.forward(u, adjs_train, device) for u in train[i:i+batch_size]], axis = 0)
          train_out = (train[0] + dt*integrating.cumsum(axis = 0)) #u(t1) = u(t0) + int{t0,t1}phi(t)dt
          train_loss = loss(train_out[:-1],train[i+1:i+batch_size])
          # backpropagation
          #torch.autograd.set_detect_anomaly(True)
          train_loss.backward()
          return train_loss
        train_loss = closure()
        optimizer.step(closure)
        rollout_train_loss.append(train_loss.item())
      #validation
      with torch.no_grad():
        integrating_valid = torch.stack([gnn.inference(u, valid_loader, device) for u in valid[0:]], axis = 0)
        valid_out = valid[0] + dt*integrating_valid.cumsum(axis = 0)
        valid_loss = loss(valid_out[:-1],valid[1:])

      mse_train = sum(rollout_train_loss)/len(rollout_train_loss)
      mse_valid = valid_loss
      # print rollout number and MSE for training and validation set at each epoch
      print(f"Rollout {t:1f}: MSE_train {mse_train :6.3f}, MSE_valid {mse_valid :6.3f}" )
      if mse_valid < mse_min:
        mse_min = mse_valid
        valid_out_best = valid_out
        torch.save(gnn, model_chk_path)
        early_stopping = 0
        print('Saving model checkpoint')
      else:
        early_stopping += 1
      #stop the training after reaching the number of epochs
      t += 1
      #import pdb; pdb.set_trace()
      if (t > epochs ): #or early_stopping == 20
        done = True

Rollout 1.000000: MSE_train  0.203, MSE_valid 29.670
Saving model checkpoint
Rollout 2.000000: MSE_train 978132.934, MSE_valid 3855132.750


KeyboardInterrupt: ignored

In [15]:
# Righe di codice per salvare l'animazione in formato .gif
import imageio
import matplotlib.pyplot as plt
def savegif(drawframe, frames, name, transparency = False, remove = True):
    filenames = []
    for i in range(frames):
        # plot frame
        drawframe(i)

        # create file name and append it to a list
        filename = f'{i}.png'
        filenames.append(filename)

        # save frame
        plt.savefig(filename, transparency = transparency)
        plt.close()
    # build gif
    with imageio.get_writer(name + '.gif', mode='I') as writer:
        for filename in filenames:
            image = imageio.imread(filename)
            writer.append_data(image)

    # Remove files
    if(remove):
        for filename in set(filenames):
            os.remove(filename)

def trajectorytogif(traj, dt, name):
    def drawframe(i):
        colorbar = plot(asfield(traj[i][:,0:2].cpu().numpy(), mesh_train))
        plt.colorbar(colorbar, shrink = 0.75)
        plt.title("T = %.2f" % (dt*i))
        plt.axis("off")
    savegif(drawframe, frames = len(traj), name = name)

In [16]:
trajectorytogif(valid_out_best, 1, name = "/images/best_prediction_1") # crea e salva la gif (la si trova nella cartella dei file generati, a sx del notebook)

# Nota: su Colab non si può, ma su jupyter notebook è invece possibile visualizzare poi la gif direttamente
# dentro il notebook, e.g.

# from  IPython.display import Image as show
# show("esempio.gif")

Calling FFC just-in-time (JIT) compiler, this may take some time.
Calling FFC just-in-time (JIT) compiler, this may take some time.
Calling FFC just-in-time (JIT) compiler, this may take some time.
Calling FFC just-in-time (JIT) compiler, this may take some time.
Calling FFC just-in-time (JIT) compiler, this may take some time.


In [ ]:
for i in range(valid_size):
  plt.figure(figsize = (12,4))
  plt.subplot(1,2,1)
  plt.title("prediction")
  plot(asfield(valid_out_best[i][:,0:2].detach().numpy()))
  plt.subplot(1,2,2)
  plt.title(f"T = {250 + i}")
  plot(asfield(train_set[50 + i][:,0:2].detach().numpy()))

In [ ]:
!git status

On branch main
Your branch is up to date with 'origin/main'.

Changes not staged for commit:
  (use "git add <file>..." to update what will be committed)
  (use "git checkout -- <file>..." to discard changes in working directory)

	modified:   Obstacle.ipynb
	modified:   gnns.py

Untracked files:
  (use "git add <file>..." to include in what will be committed)

	__pycache__/
	checkpoints/obstacle_chk.pt

no changes added to commit (use "git add" and/or "git commit -a")


In [ ]:
!git config --global user.email "filo.tombari@gmail.com"
!git config --global user.name "Filippo-Tombari"

In [ ]:
!git add .
!git commit -m "training con gpu e lbfgs"
!git push -u origin main

[main cdf140a] training con gpu e lbfgs
 6 files changed, 5 insertions(+), 3 deletions(-)
 rewrite Obstacle.ipynb (98%)
 create mode 100644 __pycache__/functional.cpython-37.pyc
 create mode 100644 __pycache__/gnns.cpython-37.pyc
 create mode 100644 __pycache__/install.cpython-37.pyc
 create mode 100644 checkpoints/obstacle_chk.pt
Counting objects: 10, done.
Delta compression using up to 2 threads.
Compressing objects: 100% (10/10), done.
Writing objects: 100% (10/10), 505.71 KiB | 12.97 MiB/s, done.
Total 10 (delta 2), reused 0 (delta 0)
remote: Resolving deltas: 100% (2/2), completed with 2 local objects.
To https://github.com/Filippo-Tombari/PdeGraph.git
   e76a7c6..cdf140a  main -> main
Branch 'main' set up to track remote branch 'main' from 'origin'.
